# 📚 Chapter 12: Using LLMs to Query Your Local Data

## 🎯 Learning Objectives

In this notebook, you will learn:

1. **Private Data Querying**: How to use GPT4All to query your private documents while maintaining data privacy
2. **Document Loading**: Loading and processing different file formats (PDF, CSV, JSON)
3. **Text Chunking**: Breaking documents into manageable chunks for LLM processing
4. **Vector Embeddings**: Using FAISS and sentence-transformers for semantic search
5. **Local LLM Querying**: Building a Q&A system with local models
6. **Code Generation for Analysis**: Using LLMs to generate Python code for data analytics

---

## 📖 Introduction

While cloud-based LLMs like OpenAI and Hugging Face Hub are powerful, **data privacy** is often a concern for businesses and developers. This chapter explores how to:

- Run LLMs **locally** using GPT4All without sending data externally
- Query **text-based data** (like PDFs) using vector embeddings
- Handle **tabular data** (CSV, JSON) by generating analytical code

> 💡 **Key Insight**: LLMs excel at text-related questions but struggle with data aggregation. For tabular data, it's better to have the LLM generate code to analyze the data rather than trying to answer directly.

---

## 🔧 Part 1: Environment Setup

### Installing Required Packages

We need several packages for this chapter:

| Package | Purpose |
|---------|--------|
| `langchain` | Chain LLM components together |
| `gpt4all` | Run models locally |
| `faiss-cpu` | Efficient similarity search |
| `sentence-transformers` | Text to vector embeddings |
| `pypdf` | Parse PDF documents |
| `jq` | Parse JSON files |

In [1]:
# Install required packages
!pip install langchain langchain-core langchain-community langchain-huggingface langchain-text-splitters -q
!pip install gpt4all -q
!pip install faiss-cpu -q
!pip install sentence-transformers -q
!pip install pypdf -q
!pip install jq -q

In [2]:
# Import all required modules
import os
import json
import warnings
warnings.filterwarnings('ignore')

# LangChain imports (updated for latest versions)
from langchain_community.document_loaders import PyPDFLoader, CSVLoader, JSONLoader
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser

print("✅ All modules imported successfully!")

✅ All modules imported successfully!


---

## 📄 Part 2: Working with PDF Documents

### 2.1 Understanding the RAG Pipeline

To query local documents with an LLM, we use a **Retrieval-Augmented Generation (RAG)** approach:

```
┌─────────────────────────────────────────────────────────────────────┐
│                        RAG Pipeline                                 │
├─────────────────────────────────────────────────────────────────────┤
│  1. Load Document → 2. Split into Chunks → 3. Create Embeddings     │
│                                                      ↓              │
│  5. Generate Answer ← 4. Find Similar Chunks ← User Question        │
└─────────────────────────────────────────────────────────────────────┘
```

### 2.2 Creating Sample Documents

Let's create some sample data for our experiments. We'll create a sample "Company Handbook" PDF simulation.

In [3]:
# Create a sample data directory
os.makedirs("./sample_data", exist_ok=True)

# For demonstration, we'll create a text file simulating document content
# In practice, you would use actual PDF files

company_handbook = """
# TechVision Inc. Employee Handbook 2024

## Chapter 1: Company Overview

TechVision Inc. was founded in 2015 in San Francisco, California. Our mission is to make
artificial intelligence accessible to everyone. We currently employ over 500 people across
12 countries.

### Core Values
- Innovation First: We encourage creative thinking and experimentation
- Customer Success: Our customers' success is our success
- Transparency: Open communication at all levels
- Sustainability: Committed to environmental responsibility

## Chapter 2: Working Hours and Leave Policy

### Standard Working Hours
Our standard working hours are 9:00 AM to 6:00 PM, Monday through Friday. We offer
flexible working arrangements for employees who need them.

### Paid Time Off (PTO)
- Annual Leave: 20 days per year for all employees
- Sick Leave: 10 days per year
- Parental Leave: 16 weeks for primary caregivers, 8 weeks for secondary caregivers
- Mental Health Days: 5 days per year

### Holidays
We observe 12 public holidays per year. Employees working on holidays receive 
double compensation or compensatory time off.

## Chapter 3: Benefits and Compensation

### Health Insurance
We provide comprehensive health insurance covering:
- Medical: 90% coverage for employees, 80% for dependents
- Dental: Full coverage for preventive care
- Vision: Annual eye exam and $300 frame allowance

### Retirement Plan
The company matches 401(k) contributions up to 6% of salary. Employees are fully
vested after 3 years of service.

### Education Benefits
- Annual learning budget: $3,000 per employee
- Conference attendance: Up to 2 conferences per year
- Tuition reimbursement: Up to $10,000 per year for approved programs

## Chapter 4: Remote Work Policy

### Hybrid Work Model
Employees can work remotely up to 3 days per week. Core collaboration hours are
10:00 AM to 3:00 PM in the employee's local timezone.

### Home Office Setup
New employees receive a $1,500 home office stipend for equipment and furniture.
Annual internet reimbursement of $50/month is provided.

### International Remote Work
Employees may work from abroad for up to 30 days per year with manager approval.
"""

# Save as a text file (in practice, this would be a PDF)
with open("./sample_data/company_handbook.txt", "w", encoding="utf-8") as f:
    f.write(company_handbook)

print("✅ Sample company handbook created!")
print(f"📄 Document length: {len(company_handbook)} characters")

✅ Sample company handbook created!
📄 Document length: 2167 characters


### 2.3 Text Chunking Explained

**Why do we need chunking?**

LLMs have a **context window limit** (e.g., 2,000-4,096 tokens). Large documents won't fit, so we must break them into smaller pieces.

```
Original Document (10,000 words)
        ↓
┌────────────────────────────────────────┐
│ Chunk 1: Words 1-500    (overlap: 50)  │ ← Each chunk fits in context window
│ Chunk 2: Words 450-950  (overlap: 50)  │ ← Overlap preserves context
│ Chunk 3: Words 900-1400 (overlap: 50)  │
│ ...                                     │
└────────────────────────────────────────┘
```

**Key Parameters:**
- `chunk_size`: Maximum characters per chunk
- `chunk_overlap`: Characters shared between consecutive chunks

In [4]:
# Read our document
with open("./sample_data/company_handbook.txt", "r", encoding="utf-8") as f:
    document_text = f.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Each chunk will be ~500 characters
    chunk_overlap=50,    # 50 characters overlap between chunks
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Split priority
)

# Split the document
chunks = text_splitter.split_text(document_text)

print(f"📊 Chunking Statistics:")
print(f"   Original document: {len(document_text)} characters")
print(f"   Number of chunks: {len(chunks)}")
print(f"\n📎 Sample Chunks:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i+1} ({len(chunk)} chars) ---")
    print(chunk[:200] + "..." if len(chunk) > 200 else chunk)

📊 Chunking Statistics:
   Original document: 2167 characters
   Number of chunks: 6

📎 Sample Chunks:

--- Chunk 1 (267 chars) ---
# TechVision Inc. Employee Handbook 2024

## Chapter 1: Company Overview

TechVision Inc. was founded in 2015 in San Francisco, California. Our mission is to make
artificial intelligence accessible to...

--- Chunk 2 (469 chars) ---
### Core Values
- Innovation First: We encourage creative thinking and experimentation
- Customer Success: Our customers' success is our success
- Transparency: Open communication at all levels
- Sust...

--- Chunk 3 (407 chars) ---
### Paid Time Off (PTO)
- Annual Leave: 20 days per year for all employees
- Sick Leave: 10 days per year
- Parental Leave: 16 weeks for primary caregivers, 8 weeks for secondary caregivers
- Mental H...


### 2.4 Understanding Embeddings

**What are embeddings?**

Embeddings convert text into numerical vectors that capture semantic meaning. Similar texts have similar vectors.

```
"I love programming"  →  [0.23, -0.45, 0.89, ...]  (384 dimensions)
"Coding is enjoyable" →  [0.21, -0.43, 0.91, ...]  ← Similar vectors!
"I hate vegetables"   →  [-0.56, 0.78, -0.22, ...] ← Different vector
```

We use the `sentence-transformers/all-MiniLM-L6-v2` model which creates 384-dimensional vectors.

In [5]:
# Initialize the embedding model
print("🔄 Loading embedding model (this may take a moment on first run)...")
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)
print("✅ Embedding model loaded!")

# Demonstrate how embeddings work
sample_texts = [
    "How many vacation days do employees get?",
    "What is the annual leave policy?",
    "Tell me about the 401k retirement plan"
]

print("\n📐 Embedding Examples:")
for text in sample_texts:
    vector = embeddings.embed_query(text)
    print(f"\n'{text}'")
    print(f"   → Vector dimensions: {len(vector)}")
    print(f"   → First 5 values: {vector[:5]}")

🔄 Loading embedding model (this may take a moment on first run)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded!

📐 Embedding Examples:

'How many vacation days do employees get?'
   → Vector dimensions: 384
   → First 5 values: [0.03040354885160923, 0.022176217287778854, 0.03267507255077362, 0.0539545863866806, -0.04998943954706192]

'What is the annual leave policy?'
   → Vector dimensions: 384
   → First 5 values: [0.04520489275455475, 0.043770406395196915, 0.014666848815977573, 0.04851733148097992, 0.10103616863489151]

'Tell me about the 401k retirement plan'
   → Vector dimensions: 384
   → First 5 values: [-0.006854142528027296, 0.15956076979637146, -0.09724941849708557, -0.0021323920227587223, 0.00961589440703392]


### 2.5 Creating a Vector Store with FAISS

**FAISS (Facebook AI Similarity Search)** is a library for efficient similarity search. It indexes vectors for fast retrieval.

```
Query: "What's the vacation policy?"
        ↓
   [embed query]
        ↓
┌─────────────────────────────────────────┐
│           FAISS Vector Store            │
│  ┌────────────────────────────────────┐ │
│  │ Chunk 1 vector ← Similarity: 0.23  │ │
│  │ Chunk 2 vector ← Similarity: 0.91 ✓│ │ ← Most similar!
│  │ Chunk 3 vector ← Similarity: 0.45  │ │
│  └────────────────────────────────────┘ │
└─────────────────────────────────────────┘
```

In [7]:
from langchain_core.documents import Document

# Convert chunks to Document objects
documents = [Document(page_content=chunk, metadata={"source": "company_handbook"}) 
             for chunk in chunks]

# Create FAISS vector store
print("🔄 Creating FAISS index...")
faiss_index = FAISS.from_documents(documents, embeddings)
print("✅ FAISS index created!")

# Save the index for later use
faiss_index.save_local("./sample_data/handbook_index")
print("💾 Index saved to './sample_data/handbook_index'")

🔄 Creating FAISS index...
✅ FAISS index created!
💾 Index saved to './sample_data/handbook_index'


In [8]:
# Demonstrate similarity search
query = "How many days of vacation do I get?"

print(f"🔍 Query: '{query}'\n")
print("📄 Top 3 Most Similar Chunks:")
print("=" * 60)

similar_docs = faiss_index.similarity_search(query, k=3)

for i, doc in enumerate(similar_docs, 1):
    print(f"\n[Match {i}]")
    print(doc.page_content)
    print("-" * 60)

🔍 Query: 'How many days of vacation do I get?'

📄 Top 3 Most Similar Chunks:

[Match 1]
### Paid Time Off (PTO)
- Annual Leave: 20 days per year for all employees
- Sick Leave: 10 days per year
- Parental Leave: 16 weeks for primary caregivers, 8 weeks for secondary caregivers
- Mental Health Days: 5 days per year

### Holidays
We observe 12 public holidays per year. Employees working on holidays receive 
double compensation or compensatory time off.

## Chapter 3: Benefits and Compensation
------------------------------------------------------------

[Match 2]
### Education Benefits
- Annual learning budget: $3,000 per employee
- Conference attendance: Up to 2 conferences per year
- Tuition reimbursement: Up to $10,000 per year for approved programs

## Chapter 4: Remote Work Policy

### Hybrid Work Model
Employees can work remotely up to 3 days per week. Core collaboration hours are
10:00 AM to 3:00 PM in the employee's local timezone.
------------------------------------------------

### 2.6 Building the Q&A System with GPT4All

Now we'll combine everything to create a complete Q&A system.

> ⚠️ **Note**: Running GPT4All requires downloading models (~4GB). The first run will download the model automatically.

In [9]:
# First, let's see available GPT4All models
from gpt4all import GPT4All

print("📋 Available GPT4All Models:")
print("=" * 50)
models = GPT4All.list_models()

# Show first 10 models
for i, model in enumerate(models[:10]):
    print(f"{i+1}. {model['filename']}")
    print(f"   Parameters: {model.get('parameters', 'N/A')}")
    print()

📋 Available GPT4All Models:
1. qwen2.5-coder-7b-instruct-q4_0.gguf
   Parameters: 8 billion

2. Meta-Llama-3-8B-Instruct.Q4_0.gguf
   Parameters: 8 billion

3. DeepSeek-R1-Distill-Qwen-7B-Q4_0.gguf
   Parameters: 7 billion

4. DeepSeek-R1-Distill-Qwen-14B-Q4_0.gguf
   Parameters: 14 billion

5. DeepSeek-R1-Distill-Llama-8B-Q4_0.gguf
   Parameters: 8 billion

6. DeepSeek-R1-Distill-Qwen-1.5B-Q4_0.gguf
   Parameters: 1.5 billion

7. Llama-3.2-3B-Instruct-Q4_0.gguf
   Parameters: 3 billion

8. Llama-3.2-1B-Instruct-Q4_0.gguf
   Parameters: 1 billion

9. Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf
   Parameters: 7 billion

10. mistral-7b-instruct-v0.1.Q4_0.gguf
   Parameters: 7 billion



In [10]:
# Download and load a GPT4All model
# Using a smaller model for faster inference
print("🔄 Loading GPT4All model (this may download ~4GB on first run)...")

# Note: Change this to a model you have or want to download
model_name = "mistral-7b-openorca.Q4_0.gguf"

try:
    # Download the model if not present
    gpt4all_model = GPT4All(model_name)
    print(f"✅ Model '{model_name}' loaded successfully!")
    MODEL_AVAILABLE = True
except Exception as e:
    print(f"⚠️ Could not load model: {e}")
    print("\n💡 You can manually download models from: https://gpt4all.io/")
    MODEL_AVAILABLE = False

🔄 Loading GPT4All model (this may download ~4GB on first run)...


Downloading: 100%|██████████| 4.11G/4.11G [00:49<00:00, 82.9MiB/s]
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


✅ Model 'mistral-7b-openorca.Q4_0.gguf' loaded successfully!


Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [11]:
# Create the Q&A function
from langchain_community.llms import GPT4All as LangChainGPT4All

# Create prompt template
qa_template = """
You are a helpful HR assistant answering questions about the employee handbook.
Use ONLY the provided context to answer. If the answer isn't in the context, 
say "I don't have that information in the handbook."

Context:
{context}

Question: {question}

Answer:"""

def ask_handbook_question(question, faiss_index, llm):
    """
    Ask a question about the company handbook.
    
    Args:
        question: The user's question
        faiss_index: The FAISS vector store
        llm: The language model
    
    Returns:
        The model's answer
    """
    # Step 1: Find relevant chunks
    relevant_docs = faiss_index.similarity_search(question, k=4)
    
    # Step 2: Combine chunks into context
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    # Step 3: Create prompt
    prompt = PromptTemplate(
        template=qa_template,
        input_variables=["context", "question"]
    ).partial(context=context)
    
    # Step 4: Create chain and invoke
    chain = prompt | llm | StrOutputParser()
    
    return chain.invoke({"question": question})

print("✅ Q&A function created!")

✅ Q&A function created!


In [12]:
# Test the Q&A system (only if model is available)
if MODEL_AVAILABLE:
    # Initialize LangChain GPT4All
    llm = LangChainGPT4All(model=model_name)
    
    # Test questions
    test_questions = [
        "How many vacation days do employees get per year?",
        "What is the 401k matching policy?",
        "Can I work remotely from another country?"
    ]
    
    print("🤖 Testing Q&A System")
    print("=" * 60)
    
    for question in test_questions:
        print(f"\n❓ Question: {question}")
        answer = ask_handbook_question(question, faiss_index, llm)
        print(f"💬 Answer: {answer}")
        print("-" * 60)
else:
    print("⚠️ Skipping Q&A test - model not available")
    print("\n💡 To test, download a GPT4All model and update the 'model_name' variable")

🤖 Testing Q&A System

❓ Question: How many vacation days do employees get per year?
💬 Answer:  Employees receive 20 days of annual leave per year.
------------------------------------------------------------

❓ Question: What is the 401k matching policy?
💬 Answer:  The company matches 401(k) contributions up to 6% of salary.
------------------------------------------------------------

❓ Question: Can I work remotely from another country?
💬 Answer:  Yes, you can work remotely from another country for up to 30 days per year with manager approval.
------------------------------------------------------------


---

## 📊 Part 3: Working with CSV Data

### 3.1 Understanding CSV Limitations with LLMs

> ⚠️ **Important**: LLMs are NOT good at analyzing tabular data directly. They excel at text understanding but struggle with aggregations and calculations.

**Why does this happen?**

When you search for "how many male passengers?", the similarity search only returns a few matching rows (not all of them), leading to incorrect counts.

In [13]:
import pandas as pd

# Create sample employee data
employee_data = {
    'employee_id': ['E001', 'E002', 'E003', 'E004', 'E005', 'E006', 'E007', 'E008'],
    'name': ['Alice Chen', 'Bob Smith', 'Carol Davis', 'David Wilson', 
             'Emma Brown', 'Frank Miller', 'Grace Lee', 'Henry Taylor'],
    'department': ['Engineering', 'Marketing', 'Engineering', 'Sales', 
                   'Engineering', 'Marketing', 'HR', 'Sales'],
    'salary': [95000, 72000, 105000, 68000, 88000, 75000, 62000, 71000],
    'hire_date': ['2020-03-15', '2019-07-22', '2018-11-08', '2021-02-14',
                  '2020-09-01', '2022-01-10', '2019-05-30', '2021-08-20'],
    'location': ['San Francisco', 'New York', 'San Francisco', 'Chicago',
                 'Austin', 'New York', 'San Francisco', 'Chicago']
}

df_employees = pd.DataFrame(employee_data)
df_employees.to_csv('./sample_data/employees.csv', index=False)

print("📊 Sample Employee Data:")
print(df_employees.to_string(index=False))

📊 Sample Employee Data:
employee_id         name  department  salary  hire_date      location
       E001   Alice Chen Engineering   95000 2020-03-15 San Francisco
       E002    Bob Smith   Marketing   72000 2019-07-22      New York
       E003  Carol Davis Engineering  105000 2018-11-08 San Francisco
       E004 David Wilson       Sales   68000 2021-02-14       Chicago
       E005   Emma Brown Engineering   88000 2020-09-01        Austin
       E006 Frank Miller   Marketing   75000 2022-01-10      New York
       E007    Grace Lee          HR   62000 2019-05-30 San Francisco
       E008 Henry Taylor       Sales   71000 2021-08-20       Chicago


In [14]:
# Load CSV using LangChain
csv_loader = CSVLoader('./sample_data/employees.csv')
csv_documents = csv_loader.load_and_split()

print(f"📄 Loaded {len(csv_documents)} document chunks from CSV")
print("\n📋 Sample document (first employee):")
print(csv_documents[0].page_content)

📄 Loaded 8 document chunks from CSV

📋 Sample document (first employee):
employee_id: E001
name: Alice Chen
department: Engineering
salary: 95000
hire_date: 2020-03-15
location: San Francisco


In [15]:
# Create embeddings for CSV
csv_texts = text_splitter.split_documents(csv_documents)
csv_faiss_index = FAISS.from_documents(csv_texts, embeddings)

# Search for engineering employees
query = "Who works in Engineering department?"
results = csv_faiss_index.similarity_search(query, k=4)

print(f"🔍 Query: '{query}'")
print("\n📄 Results (Top 4 matches):")
for i, doc in enumerate(results, 1):
    print(f"\n[Match {i}]")
    print(doc.page_content)

🔍 Query: 'Who works in Engineering department?'

📄 Results (Top 4 matches):

[Match 1]
employee_id: E001
name: Alice Chen
department: Engineering
salary: 95000
hire_date: 2020-03-15
location: San Francisco

[Match 2]
employee_id: E003
name: Carol Davis
department: Engineering
salary: 105000
hire_date: 2018-11-08
location: San Francisco

[Match 3]
employee_id: E005
name: Emma Brown
department: Engineering
salary: 88000
hire_date: 2020-09-01
location: Austin

[Match 4]
employee_id: E007
name: Grace Lee
department: HR
salary: 62000
hire_date: 2019-05-30
location: San Francisco


### 3.2 The Problem with CSV Analysis

Notice how similarity search only returns a **subset** of matching records. If we asked "How many engineers are there?", we'd get an incomplete answer.

**Solution**: Use LLMs to **generate code** for data analysis instead of querying data directly!

---

## 📋 Part 4: Working with JSON Data

JSON files are common for semi-structured data. LangChain's `JSONLoader` uses `jq` syntax to parse them.

In [16]:
# Create sample JSON data - a product catalog
product_catalog = {
    "products": [
        {
            "id": "LAPTOP-001",
            "name": "ProBook Elite 15",
            "category": "Laptops",
            "price": 1299.99,
            "specs": {
                "processor": "Intel Core i7-12700H",
                "ram": "16GB DDR5",
                "storage": "512GB NVMe SSD"
            },
            "description": "Professional laptop with excellent performance for developers and creators."
        },
        {
            "id": "PHONE-001",
            "name": "Galaxy Ultra X",
            "category": "Smartphones",
            "price": 999.99,
            "specs": {
                "display": "6.8 inch AMOLED",
                "camera": "200MP main sensor",
                "battery": "5000mAh"
            },
            "description": "Flagship smartphone with advanced camera system and all-day battery."
        },
        {
            "id": "HEADPHONES-001",
            "name": "SoundMax Pro",
            "category": "Audio",
            "price": 349.99,
            "specs": {
                "type": "Over-ear wireless",
                "noise_cancellation": "Active ANC",
                "battery_life": "40 hours"
            },
            "description": "Premium wireless headphones with industry-leading noise cancellation."
        },
        {
            "id": "TABLET-001",
            "name": "Slate Pro 12.9",
            "category": "Tablets",
            "price": 899.99,
            "specs": {
                "display": "12.9 inch Liquid Retina XDR",
                "chip": "M2 Chip",
                "storage": "256GB"
            },
            "description": "Powerful tablet for creative professionals with stunning display."
        }
    ]
}

# Save JSON file
with open('./sample_data/products.json', 'w') as f:
    json.dump(product_catalog, f, indent=2)

print("✅ Product catalog created!")
print(json.dumps(product_catalog, indent=2))

✅ Product catalog created!
{
  "products": [
    {
      "id": "LAPTOP-001",
      "name": "ProBook Elite 15",
      "category": "Laptops",
      "price": 1299.99,
      "specs": {
        "processor": "Intel Core i7-12700H",
        "ram": "16GB DDR5",
        "storage": "512GB NVMe SSD"
      },
      "description": "Professional laptop with excellent performance for developers and creators."
    },
    {
      "id": "PHONE-001",
      "name": "Galaxy Ultra X",
      "category": "Smartphones",
      "price": 999.99,
      "specs": {
        "display": "6.8 inch AMOLED",
        "camera": "200MP main sensor",
        "battery": "5000mAh"
      },
      "description": "Flagship smartphone with advanced camera system and all-day battery."
    },
    {
      "id": "HEADPHONES-001",
      "name": "SoundMax Pro",
      "category": "Audio",
      "price": 349.99,
      "specs": {
        "type": "Over-ear wireless",
        "noise_cancellation": "Active ANC",
        "battery_life": "40 hou

In [17]:
# Understanding jq schema for JSON parsing
# .products[] means: access the "products" key and iterate over each element

print("📖 Understanding jq Schema Syntax:")
print("="*50)
print("""
jq_schema='.products[]' breaks down as:

    .           → Start at root of document
    products    → Access the 'products' key
    []          → Iterate over array elements

This extracts each product as a separate document.
""")

📖 Understanding jq Schema Syntax:

jq_schema='.products[]' breaks down as:

    .           → Start at root of document
    products    → Access the 'products' key
    []          → Iterate over array elements

This extracts each product as a separate document.



In [18]:
# Load JSON with LangChain
json_loader = JSONLoader(
    file_path='./sample_data/products.json',
    jq_schema='.products[]',
    text_content=False  # Load as structured data, not just text
)

json_documents = json_loader.load_and_split()

print(f"📄 Loaded {len(json_documents)} documents from JSON")
print("\n📋 First product document:")
print(json_documents[0].page_content)

📄 Loaded 4 documents from JSON

📋 First product document:
{"id": "LAPTOP-001", "name": "ProBook Elite 15", "category": "Laptops", "price": 1299.99, "specs": {"processor": "Intel Core i7-12700H", "ram": "16GB DDR5", "storage": "512GB NVMe SSD"}, "description": "Professional laptop with excellent performance for developers and creators."}


In [19]:
# Create FAISS index for products
json_texts = text_splitter.split_documents(json_documents)
product_faiss_index = FAISS.from_documents(json_texts, embeddings)

# Query the product catalog
queries = [
    "Which products have noise cancellation?",
    "What laptop do you recommend for developers?",
    "Show me products under $500"
]

print("🔍 Product Catalog Queries:")
print("=" * 60)

for query in queries:
    print(f"\n❓ Query: {query}")
    results = product_faiss_index.similarity_search(query, k=2)
    for doc in results:
        # Extract product name from content
        print(f"   📦 {doc.page_content[:100]}...")

🔍 Product Catalog Queries:

❓ Query: Which products have noise cancellation?
   📦 {"id": "HEADPHONES-001", "name": "SoundMax Pro", "category": "Audio", "price": 349.99, "specs": {"ty...
   📦 {"id": "PHONE-001", "name": "Galaxy Ultra X", "category": "Smartphones", "price": 999.99, "specs": {...

❓ Query: What laptop do you recommend for developers?
   📦 {"id": "LAPTOP-001", "name": "ProBook Elite 15", "category": "Laptops", "price": 1299.99, "specs": {...
   📦 {"id": "TABLET-001", "name": "Slate Pro 12.9", "category": "Tablets", "price": 899.99, "specs": {"di...

❓ Query: Show me products under $500
   📦 {"id": "LAPTOP-001", "name": "ProBook Elite 15", "category": "Laptops", "price": 1299.99, "specs": {...
   📦 {"id": "TABLET-001", "name": "Slate Pro 12.9", "category": "Tablets", "price": 899.99, "specs": {"di...


---

## 🧮 Part 5: Using LLMs to Generate Analysis Code

### The Best Approach for Tabular Data

Instead of asking LLMs to analyze data directly, we ask them to **generate Python code** that we then execute.

```
┌─────────────────────────────────────────────────────────────────┐
│  Traditional Approach (Problematic)                             │
│  User Question → LLM → Wrong Answer (limited context)           │
│                                                                 │
│  Code Generation Approach (Better)                              │
│  User Question → LLM → Python Code → Execute → Correct Answer   │
└─────────────────────────────────────────────────────────────────┘
```

In [20]:
# Create a more interesting dataset for analysis
import random
from datetime import datetime, timedelta

# Generate sample sales data
np_names = ['Alice', 'Bob', 'Carol', 'David', 'Emma', 'Frank', 'Grace', 'Henry']
regions = ['North', 'South', 'East', 'West']
products = ['Widget A', 'Widget B', 'Gadget X', 'Gadget Y']

sales_data = []
base_date = datetime(2024, 1, 1)

for i in range(50):
    sale = {
        'sale_id': f'SALE-{i+1:03d}',
        'salesperson': random.choice(np_names),
        'region': random.choice(regions),
        'product': random.choice(products),
        'quantity': random.randint(1, 20),
        'unit_price': round(random.uniform(50, 200), 2),
        'date': (base_date + timedelta(days=random.randint(0, 90))).strftime('%Y-%m-%d')
    }
    sale['total'] = round(sale['quantity'] * sale['unit_price'], 2)
    sales_data.append(sale)

df_sales = pd.DataFrame(sales_data)
df_sales.to_csv('./sample_data/sales_data.csv', index=False)

print("📊 Sales Data Sample (first 10 rows):")
print(df_sales.head(10).to_string(index=False))
print(f"\n📈 Total records: {len(df_sales)}")

📊 Sales Data Sample (first 10 rows):
 sale_id salesperson region  product  quantity  unit_price       date   total
SALE-001       Frank   East Widget B         2       99.61 2024-03-16  199.22
SALE-002       David   East Widget A         3      198.45 2024-03-25  595.35
SALE-003       Alice   West Widget B        20       52.46 2024-03-12 1049.20
SALE-004       David  South Gadget Y        20       66.37 2024-02-08 1327.40
SALE-005       Alice   East Widget A        14      148.64 2024-02-15 2080.96
SALE-006       Grace   West Widget A         6      173.58 2024-01-12 1041.48
SALE-007       Alice  South Gadget X        14       69.23 2024-01-30  969.22
SALE-008       Grace   East Gadget Y        17      133.28 2024-01-07 2265.76
SALE-009       Alice  South Widget B         7      168.54 2024-03-14 1179.78
SALE-010       David  South Gadget X         7       82.82 2024-03-06  579.74

📈 Total records: 50


In [21]:
# Create the code generation prompt
code_gen_template = """
You are a Python data analyst. Given the schema of a pandas DataFrame, 
generate a SINGLE Python statement that answers the user's question.

DataFrame Schema:
- sale_id: Unique identifier (string)
- salesperson: Name of salesperson (string)
- region: Sales region - North, South, East, West (string)
- product: Product name - Widget A, Widget B, Gadget X, Gadget Y (string)
- quantity: Number of units sold (integer)
- unit_price: Price per unit (float)
- total: Total sale amount (float)
- date: Sale date in YYYY-MM-DD format (string)

The DataFrame is loaded as 'df'. Return ONLY the Python code, no explanations.

Question: {question}

Python code:"""

print("✅ Code generation template created!")

✅ Code generation template created!


In [22]:
# Function to demonstrate what the LLM might generate
# (In practice, you'd use the actual LLM)

def simulate_code_generation(question):
    """
    Simulates LLM code generation for common questions.
    In production, this would call the actual LLM.
    """
    # Map of common questions to generated pandas code
    code_mappings = {
        "total sales": "df['total'].sum()",
        "top salesperson": "df.groupby('salesperson')['total'].sum().idxmax()",
        "sales by region": "df.groupby('region')['total'].sum()",
        "average sale": "df['total'].mean()",
        "best product": "df.groupby('product')['quantity'].sum().idxmax()",
        "sales count": "len(df)"
    }
    
    for keyword, code in code_mappings.items():
        if keyword in question.lower():
            return code
    
    return "df.describe()"

# Load the DataFrame
df = pd.read_csv('./sample_data/sales_data.csv')

# Test questions
test_questions = [
    "What is the total sales amount?",
    "Who is the top salesperson by revenue?",
    "Show me sales by region",
    "What is the average sale amount?",
    "What is the best selling product?"
]

print("🧮 LLM-Generated Code Analysis Examples:")
print("=" * 60)

for question in test_questions:
    code = simulate_code_generation(question)
    result = eval(code)
    
    print(f"\n❓ Question: {question}")
    print(f"🐍 Generated Code: {code}")
    print(f"📊 Result:")
    print(result)
    print("-" * 60)

🧮 LLM-Generated Code Analysis Examples:

❓ Question: What is the total sales amount?
🐍 Generated Code: df['total'].sum()
📊 Result:
76266.76
------------------------------------------------------------

❓ Question: Who is the top salesperson by revenue?
🐍 Generated Code: df.groupby('salesperson')['total'].sum().idxmax()
📊 Result:
Grace
------------------------------------------------------------

❓ Question: Show me sales by region
🐍 Generated Code: df.groupby('region')['total'].sum()
📊 Result:
region
East     23454.21
North    12160.94
South    22711.70
West     17939.91
Name: total, dtype: float64
------------------------------------------------------------

❓ Question: What is the average sale amount?
🐍 Generated Code: df['total'].mean()
📊 Result:
1525.3352
------------------------------------------------------------

❓ Question: What is the best selling product?
🐍 Generated Code: df.describe()
📊 Result:
        quantity  unit_price        total
count  50.000000   50.000000    50.000

### 5.1 Using a Real LLM for Code Generation

In [23]:
# Complete code generation function using GPT4All
def generate_analysis_code(question, llm):
    """
    Use an LLM to generate pandas code for data analysis.
    
    Args:
        question: The user's question about the data
        llm: The language model instance
    
    Returns:
        Generated Python code string
    """
    prompt = PromptTemplate(
        template=code_gen_template,
        input_variables=["question"]
    )
    
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"question": question})
    
    # Clean the response - extract just the code
    code = response.strip()
    
    # Remove markdown code blocks if present
    if code.startswith("```python"):
        code = code[10:]
    if code.startswith("```"):
        code = code[3:]
    if code.endswith("```"):
        code = code[:-3]
    
    return code.strip()

print("✅ Code generation function created!")
print("\n💡 Usage (when model is available):")
print("   code = generate_analysis_code('What is the total sales?', llm)")
print("   result = eval(code)")

✅ Code generation function created!

💡 Usage (when model is available):
   code = generate_analysis_code('What is the total sales?', llm)
   result = eval(code)


In [26]:
# Test the code generation function with GPT4All
if MODEL_AVAILABLE:
    # Initialize the LangChain GPT4All wrapper
    from langchain_community.llms import GPT4All as LangChainGPT4All
    llm = LangChainGPT4All(model=model_name)
    
    # Load the sales DataFrame
    df = pd.read_csv('./sample_data/sales_data.csv')
    
    # Test questions
    test_questions = [
        "What is the total sales amount?",
        "Who is the top salesperson by revenue?",
        "How many sales were made in the North region?",
        "What is the average quantity sold per sale?"
    ]
    
    print("🧮 Testing LLM Code Generation")
    print("=" * 60)
    
    for question in test_questions:
        print(f"\n❓ Question: {question}")
        
        try:
            # Generate code using LLM
            generated_code = generate_analysis_code(question, llm)
            print(f"🐍 Generated Code: {generated_code}")
            
            # Try to execute the generated code
            try:
                # First try eval() for single expressions
                result = eval(generated_code)
                print(f"📊 Result: {result}")
            except SyntaxError:
                # If multi-line code, use exec() instead
                try:
                    local_vars = {'df': df}
                    exec(generated_code, globals(), local_vars)
                    # Try to get the result from local variables
                    for var_name in ['result', 'total_sales', 'answer', 'output']:
                        if var_name in local_vars:
                            print(f"📊 Result: {local_vars[var_name]}")
                            break
                    else:
                        print("📊 Code executed (check output above)")
                except Exception as exec_err:
                    print(f"⚠️ Execution error: {exec_err}")
            except Exception as eval_error:
                print(f"⚠️ Eval error: {eval_error}")
                
        except Exception as gen_error:
            print(f"⚠️ Generation error: {gen_error}")
        
        print("-" * 60)
else:
    print("⚠️ Model not available. Skipping code generation test.")

🧮 Testing LLM Code Generation

❓ Question: What is the total sales amount?
🐍 Generated Code: total_sales = df['total'].sum()
print(total_sales)
76266.76
📊 Result: 76266.76
------------------------------------------------------------

❓ Question: Who is the top salesperson by revenue?
🐍 Generated Code: top_salesperson = df[df['total'].idxmax()]
print(f"Top salesperson by revenue: {top_salesperson['salesperson']}")
⚠️ Execution error: 34
------------------------------------------------------------

❓ Question: How many sales were made in the North region?
🐍 Generated Code: len(df[df['region'] == 'North']['sale_id'].unique())
📊 Result: 8
------------------------------------------------------------

❓ Question: What is the average quantity sold per sale?
🐍 Generated Code: round(df['quantity'].mean(), 2)
📊 Result: 11.54
------------------------------------------------------------


---

## 🔄 Part 6: Putting It All Together

### Complete RAG Pipeline Class

In [29]:
class LocalDocumentQA:
    """
    A complete RAG pipeline for querying local documents with privacy.
    
    Supports:
    - PDF documents
    - CSV files
    - JSON files
    - Text files
    """
    
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2'):
        """Initialize the QA system with embedding model."""
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1024,
            chunk_overlap=64
        )
        self.index = None
        self.documents = []
        
    def add_pdf(self, file_path):
        """Add a PDF document to the knowledge base."""
        loader = PyPDFLoader(file_path)
        docs = loader.load_and_split()
        self._process_documents(docs)
        print(f"✅ Added PDF: {file_path}")
        
    def add_csv(self, file_path):
        """Add a CSV file to the knowledge base."""
        loader = CSVLoader(file_path)
        docs = loader.load_and_split()
        self._process_documents(docs)
        print(f"✅ Added CSV: {file_path}")
        
    def add_json(self, file_path, jq_schema):
        """Add a JSON file to the knowledge base."""
        loader = JSONLoader(file_path, jq_schema=jq_schema, text_content=False)
        docs = loader.load_and_split()
        self._process_documents(docs)
        print(f"✅ Added JSON: {file_path}")
        
    def add_text(self, text, metadata=None):
        """Add raw text to the knowledge base."""
        from langchain_core.documents import Document  # Fixed import
        doc = Document(page_content=text, metadata=metadata or {})
        self._process_documents([doc])
        print(f"✅ Added text ({len(text)} characters)")
        
    def _process_documents(self, docs):
        """Split documents and add to index."""
        chunks = self.text_splitter.split_documents(docs)
        self.documents.extend(chunks)
        
        # Rebuild index
        self.index = FAISS.from_documents(self.documents, self.embeddings)
        
    def search(self, query, k=4):
        """Search for relevant documents."""
        if self.index is None:
            return []
        return self.index.similarity_search(query, k=k)
    
    def save_index(self, path):
        """Save the FAISS index to disk."""
        if self.index:
            self.index.save_local(path)
            print(f"💾 Index saved to: {path}")
            
    def load_index(self, path):
        """Load a FAISS index from disk."""
        self.index = FAISS.load_local(
            path, 
            self.embeddings,
            allow_dangerous_deserialization=True
        )
        print(f"📂 Index loaded from: {path}")

print("✅ LocalDocumentQA class created!")

✅ LocalDocumentQA class created!


In [30]:
# Demonstrate the complete pipeline
print("🚀 Complete RAG Pipeline Demo")
print("=" * 60)

# Initialize the QA system
qa_system = LocalDocumentQA()

# Add various document types
qa_system.add_text(company_handbook, metadata={"source": "company_handbook"})
qa_system.add_csv('./sample_data/employees.csv')
qa_system.add_json('./sample_data/products.json', jq_schema='.products[]')

# Save the index
qa_system.save_index('./sample_data/combined_index')

print(f"\n📊 Total documents indexed: {len(qa_system.documents)}")

🚀 Complete RAG Pipeline Demo
✅ Added text (2167 characters)
✅ Added CSV: ./sample_data/employees.csv
✅ Added JSON: ./sample_data/products.json
💾 Index saved to: ./sample_data/combined_index

📊 Total documents indexed: 15


In [31]:
# Test various queries
test_queries = [
    "How many vacation days do employees get?",
    "Who works in the Engineering department?",
    "What products have good noise cancellation?",
    "What is the 401k matching policy?"
]

print("🔍 Testing Combined Knowledge Base:")
print("=" * 60)

for query in test_queries:
    print(f"\n❓ Query: {query}")
    results = qa_system.search(query, k=2)
    for i, doc in enumerate(results, 1):
        print(f"   [{i}] {doc.page_content[:150]}...")

🔍 Testing Combined Knowledge Base:

❓ Query: How many vacation days do employees get?
   [1] ### Holidays
We observe 12 public holidays per year. Employees working on holidays receive 
double compensation or compensatory time off.

## Chapter ...
   [2] # TechVision Inc. Employee Handbook 2024

## Chapter 1: Company Overview

TechVision Inc. was founded in 2015 in San Francisco, California. Our missio...

❓ Query: Who works in the Engineering department?
   [1] employee_id: E001
name: Alice Chen
department: Engineering
salary: 95000
hire_date: 2020-03-15
location: San Francisco...
   [2] employee_id: E003
name: Carol Davis
department: Engineering
salary: 105000
hire_date: 2018-11-08
location: San Francisco...

❓ Query: What products have good noise cancellation?
   [1] {"id": "HEADPHONES-001", "name": "SoundMax Pro", "category": "Audio", "price": 349.99, "specs": {"type": "Over-ear wireless", "noise_cancellation": "A...
   [2] {"id": "PHONE-001", "name": "Galaxy Ultra X", "category": "

---

## 📝 Summary

### Key Takeaways

1. **Privacy-First Approach**
   - Use GPT4All to run models locally without sending data externally
   - Embeddings are stored locally in FAISS indexes

2. **Document Processing Pipeline**
   - **Load**: Use appropriate loaders (PyPDFLoader, CSVLoader, JSONLoader)
   - **Chunk**: Split documents into manageable pieces with overlap
   - **Embed**: Convert text to vectors for semantic search
   - **Index**: Store vectors in FAISS for fast retrieval

3. **Text vs Tabular Data**
   | Data Type | Best Approach |
   |-----------|---------------|
   | Text (PDF) | Direct RAG querying |
   | Tabular (CSV, JSON) | LLM generates analysis code |

4. **When to Use What**
   - ✅ **RAG for**: Document Q&A, finding specific information, text understanding
   - ✅ **Code generation for**: Aggregations, statistics, data analysis

### 🎯 Practice Exercises

1. Add your own PDF documents and query them
2. Create a customer support FAQ system using this approach
3. Experiment with different chunk sizes and observe the impact
4. Try using different embedding models from Hugging Face

### 📚 Additional Resources

- [LangChain Documentation](https://python.langchain.com/docs/)
- [GPT4All Models](https://gpt4all.io/)
- [FAISS Documentation](https://github.com/facebookresearch/faiss)
- [Sentence Transformers](https://www.sbert.net/)

In [34]:
# Cleanup temporary files (optional)
import shutil

def cleanup():
    """Remove sample data directory."""
    if os.path.exists('./sample_data'):
        shutil.rmtree('./sample_data')
        print("🧹 Sample data cleaned up!")

# # Uncomment to clean up:
# cleanup()